In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
%load_ext aiida
%aiida
import urllib.parse

import ipywidgets as ipw
import matplotlib.pyplot as plt
from IPython.display import clear_output, display
from scipy.constants import physical_constants
import numpy as np
from surfaces_tools import utils

In [3]:
h2ev = physical_constants["Hartree energy in eV"][0]

# Replica chain

In [4]:
def print_checked_pk_list(replica_calc, check_list):
    list_of_pks = [
        replica_calc.outputs.structures[rep].pk
        for rep in replica_calc.outputs.structures
    ]
    with output_checked_pks:
        clear_output()
        print("List of all replica PKs:")
        rep_pk_str = "["
        for pk, cb in zip(list_of_pks, check_list):
            if cb.value:
                rep_pk_str += "%d " % pk
        print(rep_pk_str[:-1] + "]")


def generate_energy_cv_plot(plot_energy_scf, plot_energy_frc, plot_colvar):
    ref_en_scf = plot_energy_scf[0]
    ref_en_frc = plot_energy_frc[0]

    plot_energy_scf = np.array(plot_energy_scf) * h2ev
    plot_energy_scf -= ref_en_scf * h2ev

    plot_energy_frc = np.array(plot_energy_frc) * h2ev
    plot_energy_frc -= ref_en_frc * h2ev

    plt.figure(figsize=(10, 5))
    plt.ylabel("Energy/eV")
    plt.xlabel("Distance from replica 0 (Å)")
    plt.plot(plot_colvar, plot_energy_scf, "go-", label="SCF energy")
    plt.plot(plot_colvar, plot_energy_frc, "bo-", label="FORCE_EVAL energy")

    #    for i, rep in enumerate(replica_calc["replicas"]):
    #        # if the replica has no cv_target and has energy, it's an initial one, paint as red
    #        cv_target, energy, struct_pk = rep
    #        if cv_target == None and energy is not None:
    #            plt.plot(plot_colvar[i], plot_energy_scf[i], "ro-")

    plt.legend()
    plt.grid()
    plt.show()


def generate_thumbnail_html(wc):
    html_list = []
    check_list = []  # check boxes to show in final pk list

    plot_energy_scf = []
    plot_energy_frc = []
    plot_colvar = []

    dist = 0
    for rep in wc.outputs.structures:
        struct_node = wc.outputs.structures[rep]
        cvs_target = wc.outputs.details[rep]["cvs_target"]
        cvs_actual = wc.outputs.details[rep]["cvs_actual"]
        try:
            energy = [
                wc.outputs.details[rep]["output_parameters"]["energy_scf"],
                wc.outputs.details[rep]["output_parameters"]["energy"],
            ]
        except KeyError:
            energy = [
                wc.outputs.details[rep]["output_parameters"]["energy_scf"],
                wc.outputs.details[rep]["output_parameters"]["energy_scf"],
            ]
            
        plot_energy_scf.append(energy[0])
        plot_energy_frc.append(energy[1])

        html = '<table style="border-spacing:20px 0px;border-collapse:separate;">'

        thumbnail = utils.ase_to_thumbnail(struct_node.get_ase())

        cv_target_str = " ".join(["%.2f" % val for val in cvs_target])

        energy_scf_str = "%.6f" % energy[0]
        energy_frc_str = "%.6f" % energy[1]

        colvar_actual_str = " ".join(["%.2f" % val for val in cvs_actual])
        d2prev = wc.outputs.details[rep]["d2prev"]
        dist += d2prev
        plot_colvar.append(dist)
        d2prev_str = "%.4f" % float(d2prev)

        check_me = ipw.Checkbox(
            value=True, description="select", disabled=False, layout=layout
        )
        check_me.observe(
            lambda x, rc=wc, cl=check_list: print_checked_pk_list(rc, cl),
            "value",
        )
        check_list.append(check_me)

        html = '<img width="400px" src="data:image/png;base64,{}" title="">'.format(
            thumbnail
        )

        html += '<table style="border-spacing:6px 0px;border-collapse:separate;">'

        # Output some information about the replica...
        html += '<tr><td align="left"><b>Target:</b></td><td align="right">{}</td></tr>'.format(
            cv_target_str
        )
        html += '<tr><td align="left"><b>CV actual:</b></td><td align="right">{}</td></tr>'.format(
            colvar_actual_str
        )
        html += '<tr><td align="left"><b>Energy (scf) /au:</b></td><td align="right">{}</td></tr>'.format(
            energy_scf_str
        )
        html += '<tr><td align="left"><b>Energy (force) /au:</b></td><td align="right">{}</td></tr>'.format(
            energy_frc_str
        )
        html += '<tr><td align="left"><b>dist to prev:</b></td><td align="right">{}</td></tr>'.format(
            d2prev_str
        )
        html += '<tr><td align="left">pk: </td><td align="right">{}</td></tr>'.format(
            struct_node.pk
        )

        # ... and the download link.
        html += '<tr><td align="left"><a target="_blank" href="export_structure.ipynb?uuid={}">View & export</a></td></tr>'.format(
            struct_node.uuid
        )

        html += "</table>"

        html_list.append(html)

    return html_list, check_list, plot_energy_scf, plot_energy_frc, plot_colvar

In [5]:
def process_and_show_replica(_=None):
    wc = load_node(node.value)
    (
        html_list,
        check_list,
        plot_energy_scf,
        plot_energy_frc,
        plot_colvar,
    ) = generate_thumbnail_html(wc)

    with output_plot:
        clear_output()
        generate_energy_cv_plot(plot_energy_scf, plot_energy_frc, plot_colvar)

    n_col = 4
    layout = {"width": "%.1f%%" % (100.0 / n_col)}
    with output_thumbnails:
        clear_output()
        cur_row = []
        for i, (html, cb) in enumerate(zip(html_list, check_list)):
            cur_row.append(ipw.VBox([ipw.HTML(html), cb], layout=layout))
            if (i + 1) % n_col == 0:
                display(ipw.HBox(cur_row))
                cur_row = []
        if len(cur_row) != 0:
            display(ipw.HBox(cur_row))

    print_checked_pk_list(wc, check_list)

In [6]:
style = {"description_width": "120px"}
layout = {"width": "70%"}

node = ipw.IntText(description="Load node: ", layout=layout, style=style)

btn_show = ipw.Button(description="Show")
btn_show.on_click(process_and_show_replica)

# output_header = ipw.Output()
output_plot = ipw.Output()
output_thumbnails = ipw.Output()
output_checked_pks = ipw.Output()

display(node, btn_show, output_plot, output_thumbnails, output_checked_pks)

IntText(value=0, description='Load node: ', layout=Layout(width='70%'), style=DescriptionStyle(description_wid…

Button(description='Show', style=ButtonStyle())

Output()

Output()

Output()

In [7]:
# Load the URL after everything is set up.
try:
    url = urllib.parse.urlsplit(jupyter_notebook_url)
    node.value = urllib.parse.parse_qs(url.query)["pk"][0]
    process_and_show_replica(0)
except Exception as exc:
    print(exc)